Great 👍 let’s extend the framework with **Memory** so your Parent Agent + Child Agents can store and reuse context.

---

# 🔹 Types of Memory in LangChain/LangGraph

1. **Short-Term Memory** → keeps state only for the session (conversation buffer).
   Example: remembering what the user just asked in this workflow.

2. **Long-Term Memory** → persists context in a vector DB (FAISS, Chroma, Pinecone, etc.).
   Example: remembering past weather events and their fulfillment impact.

We’ll combine **ConversationBufferMemory** + **VectorStoreRetrieverMemory**.

---

# 🔹 Updated Python Framework with Memory

```python
from langgraph.graph import StateGraph, START, END
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# -------------------------
# Parent State (with memory hooks)
# -------------------------
class ParentState(dict):
    query: str
    context: dict
    actions: list
    result: str

# -------------------------
# Setup Memory
# -------------------------
# Short-term memory (per conversation/session)
short_term_memory = ConversationBufferMemory(memory_key="chat_history")

# Long-term memory (persistent knowledge store)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("memory_index", embeddings) if False else FAISS.from_texts(
    ["Storm in 2023 delayed fulfillment by 2 days",
     "Apple Pay implementation required PCI compliance changes"],
    embeddings
)
retriever = vectorstore.as_retriever()

# -------------------------
# Child Agents (Tools)
# -------------------------
def weather_agent(state):
    info = "Storm in Texas, impacting Dallas region"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"weather": info}

def inventory_agent(state):
    info = "Alternate locations: Austin Store, Houston Store"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"alternate_locations": ["Austin Store", "Houston Store"]}

def order_agent(state):
    info = "Impacted orders: ORD123, ORD456 (requires approval)"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"impacted_orders": ["ORD123", "ORD456"], "requires_approval": True}

def fulfillment_agent(state):
    info = "Fulfillment Manager notified"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"action": info}

def store_info_agent(state):
    info = "Austin Store has stock available"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"store_details": {"Austin Store": "Stock available"}}

def jira_agent(state):
    return {"tickets": ["JIRA-101", "JIRA-102"]}

def confluence_agent(state):
    return {"docs": ["ApplePay_DesignDoc"]}

def github_agent(state):
    return {"commits": ["commit123", "commit456"]}

# -------------------------
# Graph Builder
# -------------------------
builder = StateGraph(ParentState)

# Add nodes (Child Agents as tools)
builder.add_node("weather_node", weather_agent)
builder.add_node("inventory_node", inventory_agent)
builder.add_node("order_node", order_agent)
builder.add_node("fulfillment_node", fulfillment_agent)
builder.add_node("store_node", store_info_agent)
builder.add_node("jira_node", jira_agent)
builder.add_node("confluence_node", confluence_agent)
builder.add_node("github_node", github_agent)

# Define flow for Use Case 1 (Weather Impact)
builder.add_edge(START, "weather_node")
builder.add_edge("weather_node", "inventory_node")
builder.add_edge("inventory_node", "order_node")
builder.add_edge("order_node", "fulfillment_node")
builder.add_edge("fulfillment_node", "store_node")
builder.add_edge("store_node", END)

# Define flow for Use Case 2 (Apple Pay Summary)
builder.add_edge(START, "jira_node")
builder.add_edge("jira_node", "confluence_node")
builder.add_edge("confluence_node", "github_node")
builder.add_edge("github_node", END)

graph = builder.compile()

# -------------------------
# Run Examples
# -------------------------
result1 = graph.invoke({"query": "What is the impact of weather on order fulfillment?"})
print("Use Case 1 Result:", result1)
print("\n🔹 Short-Term Memory:\n", short_term_memory.load_memory_variables({}))

# Retrieve similar past knowledge from long-term memory
docs = retriever.get_relevant_documents("weather impact on fulfillment")
print("\n🔹 Long-Term Memory Retrieved:\n", [d.page_content for d in docs])

result2 = graph.invoke({"query": "Explain Apple Pay implementation with tickets and commits"})
print("\nUse Case 2 Result:", result2)
```

---

# 🔹 What Changed?

✅ **Short-Term Memory**

* Every child agent logs its input/output into `ConversationBufferMemory`.
* Parent can recall recent steps → improves reasoning.

✅ **Long-Term Memory (Vector DB)**

* FAISS/Chroma stores embeddings of past events.
* Example: Past storm events help predict current delays.
* Developer queries (e.g., Apple Pay) can fetch past Confluence or Jira summaries.

✅ **Retrieval + RAG**

* Parent Agent can now **enrich reasoning** using retriever context.

---

⚡This makes the AI **multi-session aware**:

* **Session memory (short-term)** = “Remembers this conversation flow.”
* **Knowledge memory (long-term)** = “Knows past fulfillment delays or Apple Pay implementations.”

---

Would you like me to **extend this into a reusable framework class** (so you can just plug in new use cases, agents, and memory modules without rewriting the whole graph)?


In [1]:
from langgraph.graph import StateGraph, START, END
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# -------------------------
# Parent State (with memory hooks)
# -------------------------
class ParentState(dict):
    query: str
    context: dict
    actions: list
    result: str

# -------------------------
# Setup Memory
# -------------------------
# Short-term memory (per conversation/session)
short_term_memory = ConversationBufferMemory(memory_key="chat_history")

# Long-term memory (persistent knowledge store)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("memory_index", embeddings) if False else FAISS.from_texts(
    ["Storm in 2023 delayed fulfillment by 2 days",
     "Apple Pay implementation required PCI compliance changes"],
    embeddings
)
retriever = vectorstore.as_retriever()

# -------------------------
# Child Agents (Tools)
# -------------------------
def weather_agent(state):
    info = "Storm in Texas, impacting Dallas region"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"weather": info}

def inventory_agent(state):
    info = "Alternate locations: Austin Store, Houston Store"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"alternate_locations": ["Austin Store", "Houston Store"]}

def order_agent(state):
    info = "Impacted orders: ORD123, ORD456 (requires approval)"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"impacted_orders": ["ORD123", "ORD456"], "requires_approval": True}

def fulfillment_agent(state):
    info = "Fulfillment Manager notified"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"action": info}

def store_info_agent(state):
    info = "Austin Store has stock available"
    short_term_memory.save_context({"input": state["query"]}, {"output": info})
    return {"store_details": {"Austin Store": "Stock available"}}

def jira_agent(state):
    return {"tickets": ["JIRA-101", "JIRA-102"]}

def confluence_agent(state):
    return {"docs": ["ApplePay_DesignDoc"]}

def github_agent(state):
    return {"commits": ["commit123", "commit456"]}

# -------------------------
# Graph Builder
# -------------------------
builder = StateGraph(ParentState)

# Add nodes (Child Agents as tools)
builder.add_node("weather_node", weather_agent)
builder.add_node("inventory_node", inventory_agent)
builder.add_node("order_node", order_agent)
builder.add_node("fulfillment_node", fulfillment_agent)
builder.add_node("store_node", store_info_agent)
builder.add_node("jira_node", jira_agent)
builder.add_node("confluence_node", confluence_agent)
builder.add_node("github_node", github_agent)

# Define flow for Use Case 1 (Weather Impact)
builder.add_edge(START, "weather_node")
builder.add_edge("weather_node", "inventory_node")
builder.add_edge("inventory_node", "order_node")
builder.add_edge("order_node", "fulfillment_node")
builder.add_edge("fulfillment_node", "store_node")
builder.add_edge("store_node", END)

# Define flow for Use Case 2 (Apple Pay Summary)
builder.add_edge(START, "jira_node")
builder.add_edge("jira_node", "confluence_node")
builder.add_edge("confluence_node", "github_node")
builder.add_edge("github_node", END)

graph = builder.compile()

# -------------------------
# Run Examples
# -------------------------
result1 = graph.invoke({"query": "What is the impact of weather on order fulfillment?"})
print("Use Case 1 Result:", result1)
print("\n🔹 Short-Term Memory:\n", short_term_memory.load_memory_variables({}))

# Retrieve similar past knowledge from long-term memory
docs = retriever.get_relevant_documents("weather impact on fulfillment")
print("\n🔹 Long-Term Memory Retrieved:\n", [d.page_content for d in docs])

result2 = graph.invoke({"query": "Explain Apple Pay implementation with tickets and commits"})
print("\nUse Case 2 Result:", result2)


/var/folders/1y/n1s876vs41n5j660qsm8btcm0000gn/T/ipykernel_20338/738663202.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  short_term_memory = ConversationBufferMemory(memory_key="chat_history")
/var/folders/1y/n1s876vs41n5j660qsm8btcm0000gn/T/ipykernel_20338/738663202.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error